## Please complete the following tasks

Let's go through the following tasks so that we understand how the code works:
1. Get station locations and bike availability data.
2. Convert the data to a dataframe
3. Write a program to display all of the stations on a map.
4. See if you can modify the size of the marker based on the number of bikes available at the station.

Here is the Divvy data [instructions](https://www.divvybikes.com/system-data).
Here is a useful [python dictionary formatter](https://codebeautify.org/python-formatter-beautifier).

In [ ]:
# import libraries:
from urllib.request import urlopen
import pprint
import json
import pandas as pd

## Let's Analyze the Bike Data

### 1a. Load the Station Locations

In [ ]:
response = urlopen('https://gbfs.divvybikes.com/gbfs/en/station_information.json')
results = response.read().decode('utf-8', 'ignore')
station_info = json.loads(results)
station_list = station_info['data']['stations']
# print(station_list)

### 1b. Load the Station Availability

In [ ]:
response = urlopen('https://gbfs.divvybikes.com/gbfs/en/station_status.json')
results = response.read().decode('utf-8', 'ignore')
station_availability = json.loads(results)
bike_availability_by_station = station_availability['data']['stations']
# print(bike_availability_by_station)

### 1c. Create a data frame from the dictionary

In [ ]:
df_station_locations = pd.DataFrame(station_list).set_index('station_id')
df_station_locations.head(5)

### 1b. Loading Availability

In [ ]:
df_station_availability = pd.DataFrame(bike_availability_by_station).set_index('station_id')
df_station_availability.head(5)

### 1c. Joining Location and Availability Tables

In [ ]:
merged_table = df_station_availability.join(df_station_locations)
simplified_table = merged_table[["name", "lat", "lon", "num_bikes_available", "num_docks_available"]]
simplified_table.head(10)

### 2. Filtering and Sorting
Modify the dataframe so that the station with the most num_bikes_available is at the top.

In [ ]:
# your code here
simplified_table.sort_values('num_bikes_available', ascending=False)
simplified_table.head(10)

### 3. Filtering Data with User Input
Write a program that prompts the user for a station name and then displays any station that has a partial match.

In [ ]:
# 1. Write a program that prompts the user for a 
# station name and then displays any station that has a partial match.

# Note: this syntax is not straightforward. Found it on this forum: 
# https://stackoverflow.com/questions/27975069/how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe
search_term = input('Which station are you looking for? ')
my_filter = simplified_table['name'].str.contains(search_term)
filtered_df = simplified_table[my_filter]
filtered_df



### 4. Display All of the Stations on the Map

In [ ]:
import folium

# create map:
chicago_map = folium.Map(
    location=[41.867226, -87.615355],  #lat, lng
    zoom_start=12,
    tiles="Stamen toner"  #"Stamen watercolor"
)

for index, station in simplified_table.head(100).iterrows():
    circle_marker = folium.CircleMarker(
        [station.get('lat'), station.get('lon')], 
        radius=station.get('num_bikes_available') * 5, 
        fill_color='teal', 
        color='teal', 
        popup=station.get('name') + ' has ' + str(station.get('num_bikes_available')) + ' available bikes.'
    )
    circle_marker.add_to(chicago_map)



# show map:
chicago_map